In [ ]:
using Revise
using OnePiece
using GSEA

In [ ]:
da = joinpath(@__DIR__, "sarcopenia")

In [ ]:
sett = joinpath(da, "setting.json")

In [ ]:
te = joinpath(tempdir(), "GSEA")

rm(te, recursive=true)

mkdir(te)

In [ ]:
ke_ar = OnePiece.dict.read(sett)

In [ ]:
GSEA.metric_rank(
    sett,
    joinpath(da, "set_genes.json"),
    joinpath(da, "target_x_sample_x_number.tsv"),
    joinpath(da, "gene_x_sample_x_score.tsv"),
    te,
)

In [ ]:
be = "$(ke_ar["permutation"])_$(ke_ar["number_of_permutations"])"

## Metric

In [ ]:
ol_ge_st =
    OnePiece.table.read(joinpath(da, be, "ranked_gene_list_0_versus_1.tsv"))[!, ["NAME", "SCORE"]]

sc_ge_st = OnePiece.table.read(joinpath(te, "gene_x_metric_x_score.tsv"))

In [ ]:
function ro(re)

    round(re, digits = 4)

end

In [ ]:
fe_sc = Dict(fe => ro(sc) for (fe, sc) in eachrow(sc_ge_st))

println("Gene\tOld\tNew")

n_mi = 0

for (fe, ol) in Dict(fe => ro(sc) for (fe, sc) in eachrow(ol_ge_st))

    sc = fe_sc[fe]

    if ol != sc

        println("$fe\t$ol\t$sc")

        global n_mi += 1

    end

end

n_fe = size(ol_ge_st, 1)

mi = n_mi / n_fe

println("Missed $n_mi/$n_fe scores ($(ro(mi * 100))%).")

@assert mi < 0.001

## Statistics

In [ ]:
function re(id)

    OnePiece.table.read(joinpath(da, be, "gsea_report_for_$id.tsv"))

end

In [ ]:
ol_se_st = vcat((re(id) for id in [0, 1])...)

size(ol_se_st)

In [ ]:
va_se_st = OnePiece.table.read(joinpath(te, "set_x_statistic_x_number.tsv"))

size(va_se_st)

In [ ]:
se_ = ol_se_st[!, 1]

@assert isempty(symdiff(se_, va_se_st[!, 1]))

In [ ]:
va_se_st = va_se_st[indexin(se_, va_se_st[!, 1]), :]

for (nao, nan) in (
    ("ES", "Enrichment"),
    ("NES", "Enrichment"),
    ("NOM p-val", "Global pvalue"),
    ("FDR q-val", "Adjusted global pvalue"),
)

    OnePiece.figure.plot_x_y(
        [ol_se_st[!, nao]],
        [va_se_st[!, nan]],
        text_ = [se_],
        mode_ = ["markers"],
        la = Dict(
            "title" => be,
            "xaxis" => Dict("title" => Dict("text" => nao)),
            "yaxis" => Dict("title" => Dict("text" => nan)),
        ),
        ou = joinpath(te, "$(OnePiece.path.clean(nao)).html"),
    )

end